In [1]:
import cv2
import numpy as np
import os
import sys
import dlib
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
#use SHORT_TEST for fast testing of only one entry
SHORT_TEST = True
detector = dlib.get_frontal_face_detector()
f="shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(f)

In [3]:
def get_rect(shape):
    rw = 0
    rh = 0
    rx = 65535
    ry = 65535
    for (x,y) in shape:
        rw = max(rw,x)
        rh = max(rh,y)
        rx = min(rx,x)
        ry = min(ry,y)
    return (rx,ry,rw-rx,rh-ry)
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords


In [14]:
inpDir='Dataset'
person = {}
img = None
mouth = None
firstImg = None
firstPerson = None
firstSentenceId = None
firstSentenceId2 = None
ratio = 1.0/2.0
pi = 0
linpDir = os.listdir(inpDir)
for personStr in linpDir:
    pi += 1
    print("person: %s [%d/%d]" % (personStr,pi,len(linpDir)))
    person[personStr] = {}
    personFolder = '%s/%s/phrases' % (inpDir,personStr)
    lpersonFolder = os.listdir(personFolder)
    si = 0
    for sentenceId in lpersonFolder:
        si += 1
        c=0
        person[personStr][sentenceId] = {}
        print("reading sentence %s for person %s [%d/%d,%d/%d]" % (sentenceId,personStr,si,len(lpersonFolder),pi,len(linpDir)))
        sentenceFolder = '%s/%s' % (personFolder,sentenceId)
        for sentenceId2 in os.listdir(sentenceFolder):
            sentenceFolder2 = '%s/%s/%s' % (personFolder,sentenceId,sentenceId2)
            person[personStr][sentenceId][sentenceId2] = {}
            c=c+1
            print(c)
            for frame in os.listdir(sentenceFolder2):
                file = "%s/%s" % (sentenceFolder2,frame)
                if(not os.path.isfile(file)):
                    print("%s does not exist" % file)
                    sys.exit(1)
                if(frame[0:5] != "color"):
                    # skip depth data
                    # kinda reminds me on the song "War" of Edwin Starr
                    continue
                frame = frame[6:-4]
                #print("reading file: %s" % file)
                img = cv2.imread(file)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                #speed up detector by resizing first
                img2 = cv2.resize(img, None, fx=ratio, fy=ratio)

                
                #detects whole face
                rects = detector(img2, 1)
                if len(rects) == 0:
                    print("error finding head at file: %s" % file)
                    continue
                rects[0] = dlib.scale_rect(rects[0],1/ratio)
                shape = predictor(image=img,box=rects[0])
                shape = shape_to_np(shape)
                
                #indices from 48 to 67 are for mouth
                shape = [shape[x] for x in range(48,68)]
                (x, y, w, h) = get_rect(shape)

                for s in shape:
                    s[0] -= x
                    s[1] -= y
                
                person[personStr][sentenceId][sentenceId2][frame] = {}
                if firstImg is None:
                    firstImg = img
                    firstPerson = personStr
                    firstSentenceId = sentenceId
                    firstSentenceId2 = sentenceId2
                if ((firstPerson == personStr) and (firstSentenceId == sentenceId) and (firstSentenceId2 == sentenceId2)):
                    mouth = img[y:y+h, x:x+w].copy()
                    
                
                
                print("filling: person[%s][%s][%s][%s] with shapes %d" % (personStr,sentenceId,sentenceId2,frame,len(shape)))
                person[personStr][sentenceId][sentenceId2][frame]["mouth"] = mouth
                
                person[personStr][sentenceId][sentenceId2][frame]["shape"] = shape
            
        #for now just one sentence
       
        #if SHORT_TEST:
            #break
    #for now just one person
    #if SHORT_TEST:
        #break

person: F01 [1/15]
reading sentence 01 for person F01 [1/10,1/15]
1
filling: person[F01][01][01][001] with shapes 20
filling: person[F01][01][01][002] with shapes 20
filling: person[F01][01][01][003] with shapes 20
filling: person[F01][01][01][004] with shapes 20
filling: person[F01][01][01][005] with shapes 20
filling: person[F01][01][01][006] with shapes 20
filling: person[F01][01][01][007] with shapes 20
filling: person[F01][01][01][008] with shapes 20
filling: person[F01][01][01][009] with shapes 20
filling: person[F01][01][01][010] with shapes 20
filling: person[F01][01][01][011] with shapes 20
filling: person[F01][01][01][012] with shapes 20
2
filling: person[F01][01][02][001] with shapes 20
filling: person[F01][01][02][002] with shapes 20
filling: person[F01][01][02][003] with shapes 20
filling: person[F01][01][02][004] with shapes 20
filling: person[F01][01][02][005] with shapes 20
filling: person[F01][01][02][006] with shapes 20
filling: person[F01][01][02][007] with shapes 20

5
filling: person[F01][02][05][001] with shapes 20
filling: person[F01][02][05][002] with shapes 20
filling: person[F01][02][05][003] with shapes 20
filling: person[F01][02][05][004] with shapes 20
filling: person[F01][02][05][005] with shapes 20
filling: person[F01][02][05][006] with shapes 20
filling: person[F01][02][05][007] with shapes 20
filling: person[F01][02][05][008] with shapes 20
filling: person[F01][02][05][009] with shapes 20
filling: person[F01][02][05][010] with shapes 20
filling: person[F01][02][05][011] with shapes 20
filling: person[F01][02][05][012] with shapes 20
6
filling: person[F01][02][06][001] with shapes 20
filling: person[F01][02][06][002] with shapes 20
filling: person[F01][02][06][003] with shapes 20
filling: person[F01][02][06][004] with shapes 20
filling: person[F01][02][06][005] with shapes 20
filling: person[F01][02][06][006] with shapes 20
filling: person[F01][02][06][007] with shapes 20
filling: person[F01][02][06][008] with shapes 20
filling: person[

filling: person[F01][04][01][006] with shapes 20
2
filling: person[F01][04][02][001] with shapes 20
filling: person[F01][04][02][002] with shapes 20
filling: person[F01][04][02][003] with shapes 20
filling: person[F01][04][02][004] with shapes 20
filling: person[F01][04][02][005] with shapes 20
filling: person[F01][04][02][006] with shapes 20
filling: person[F01][04][02][007] with shapes 20
filling: person[F01][04][02][008] with shapes 20
filling: person[F01][04][02][009] with shapes 20
filling: person[F01][04][02][010] with shapes 20
3
filling: person[F01][04][03][001] with shapes 20
filling: person[F01][04][03][002] with shapes 20
filling: person[F01][04][03][003] with shapes 20
filling: person[F01][04][03][004] with shapes 20
filling: person[F01][04][03][005] with shapes 20
filling: person[F01][04][03][006] with shapes 20
filling: person[F01][04][03][007] with shapes 20
filling: person[F01][04][03][008] with shapes 20
4
filling: person[F01][04][04][001] with shapes 20
filling: perso

reading sentence 06 for person F01 [6/10,1/15]
1
filling: person[F01][06][01][001] with shapes 20
filling: person[F01][06][01][002] with shapes 20
filling: person[F01][06][01][003] with shapes 20
filling: person[F01][06][01][004] with shapes 20
filling: person[F01][06][01][005] with shapes 20
filling: person[F01][06][01][006] with shapes 20
filling: person[F01][06][01][007] with shapes 20
filling: person[F01][06][01][008] with shapes 20
filling: person[F01][06][01][009] with shapes 20
2
filling: person[F01][06][02][001] with shapes 20
filling: person[F01][06][02][002] with shapes 20
filling: person[F01][06][02][003] with shapes 20
filling: person[F01][06][02][004] with shapes 20
filling: person[F01][06][02][005] with shapes 20
filling: person[F01][06][02][006] with shapes 20
filling: person[F01][06][02][007] with shapes 20
filling: person[F01][06][02][008] with shapes 20
filling: person[F01][06][02][009] with shapes 20
filling: person[F01][06][02][010] with shapes 20
3
filling: person[

filling: person[F01][07][07][007] with shapes 20
filling: person[F01][07][07][008] with shapes 20
filling: person[F01][07][07][009] with shapes 20
filling: person[F01][07][07][010] with shapes 20
8
filling: person[F01][07][08][001] with shapes 20
filling: person[F01][07][08][002] with shapes 20
filling: person[F01][07][08][003] with shapes 20
filling: person[F01][07][08][004] with shapes 20
filling: person[F01][07][08][005] with shapes 20
filling: person[F01][07][08][006] with shapes 20
filling: person[F01][07][08][007] with shapes 20
filling: person[F01][07][08][008] with shapes 20
filling: person[F01][07][08][009] with shapes 20
9
filling: person[F01][07][09][001] with shapes 20
filling: person[F01][07][09][002] with shapes 20
filling: person[F01][07][09][003] with shapes 20
filling: person[F01][07][09][004] with shapes 20
filling: person[F01][07][09][005] with shapes 20
filling: person[F01][07][09][006] with shapes 20
filling: person[F01][07][09][007] with shapes 20
filling: person[

...

filling: person[M08][04][07][002] with shapes 20
filling: person[M08][04][07][003] with shapes 20
filling: person[M08][04][07][004] with shapes 20
filling: person[M08][04][07][005] with shapes 20
filling: person[M08][04][07][006] with shapes 20
filling: person[M08][04][07][007] with shapes 20
filling: person[M08][04][07][008] with shapes 20
8
filling: person[M08][04][08][001] with shapes 20
filling: person[M08][04][08][002] with shapes 20
filling: person[M08][04][08][003] with shapes 20
filling: person[M08][04][08][004] with shapes 20
filling: person[M08][04][08][005] with shapes 20
filling: person[M08][04][08][006] with shapes 20
filling: person[M08][04][08][007] with shapes 20
filling: person[M08][04][08][008] with shapes 20
filling: person[M08][04][08][009] with shapes 20
9
filling: person[M08][04][09][001] with shapes 20
filling: person[M08][04][09][002] with shapes 20
filling: person[M08][04][09][003] with shapes 20
filling: person[M08][04][09][004] with shapes 20
filling: person[

filling: person[M08][06][05][001] with shapes 20
filling: person[M08][06][05][002] with shapes 20
filling: person[M08][06][05][003] with shapes 20
filling: person[M08][06][05][004] with shapes 20
filling: person[M08][06][05][005] with shapes 20
filling: person[M08][06][05][006] with shapes 20
filling: person[M08][06][05][007] with shapes 20
filling: person[M08][06][05][008] with shapes 20
filling: person[M08][06][05][009] with shapes 20
6
filling: person[M08][06][06][001] with shapes 20
filling: person[M08][06][06][002] with shapes 20
filling: person[M08][06][06][003] with shapes 20
filling: person[M08][06][06][004] with shapes 20
filling: person[M08][06][06][005] with shapes 20
filling: person[M08][06][06][006] with shapes 20
filling: person[M08][06][06][007] with shapes 20
filling: person[M08][06][06][008] with shapes 20
filling: person[M08][06][06][009] with shapes 20
7
filling: person[M08][06][07][001] with shapes 20
filling: person[M08][06][07][002] with shapes 20
filling: person[

filling: person[M08][07][09][013] with shapes 20
10
filling: person[M08][07][10][001] with shapes 20
filling: person[M08][07][10][002] with shapes 20
filling: person[M08][07][10][003] with shapes 20
filling: person[M08][07][10][004] with shapes 20
filling: person[M08][07][10][005] with shapes 20
filling: person[M08][07][10][006] with shapes 20
filling: person[M08][07][10][007] with shapes 20
filling: person[M08][07][10][008] with shapes 20
filling: person[M08][07][10][009] with shapes 20
filling: person[M08][07][10][010] with shapes 20
filling: person[M08][07][10][011] with shapes 20
filling: person[M08][07][10][012] with shapes 20
filling: person[M08][07][10][013] with shapes 20
reading sentence 08 for person M08 [8/10,15/15]
1
filling: person[M08][08][01][001] with shapes 20
filling: person[M08][08][01][002] with shapes 20
filling: person[M08][08][01][003] with shapes 20
filling: person[M08][08][01][004] with shapes 20
filling: person[M08][08][01][005] with shapes 20
filling: person[

filling: person[M08][09][02][007] with shapes 20
filling: person[M08][09][02][008] with shapes 20
filling: person[M08][09][02][009] with shapes 20
filling: person[M08][09][02][010] with shapes 20
filling: person[M08][09][02][011] with shapes 20
filling: person[M08][09][02][012] with shapes 20
filling: person[M08][09][02][013] with shapes 20
filling: person[M08][09][02][014] with shapes 20
filling: person[M08][09][02][015] with shapes 20
3
filling: person[M08][09][03][001] with shapes 20
filling: person[M08][09][03][002] with shapes 20
filling: person[M08][09][03][003] with shapes 20
filling: person[M08][09][03][004] with shapes 20
filling: person[M08][09][03][005] with shapes 20
filling: person[M08][09][03][006] with shapes 20
filling: person[M08][09][03][007] with shapes 20
filling: person[M08][09][03][008] with shapes 20
filling: person[M08][09][03][009] with shapes 20
4
filling: person[M08][09][04][001] with shapes 20
filling: person[M08][09][04][002] with shapes 20
filling: person[

filling: person[M08][10][07][004] with shapes 20
filling: person[M08][10][07][005] with shapes 20
filling: person[M08][10][07][006] with shapes 20
filling: person[M08][10][07][007] with shapes 20
filling: person[M08][10][07][008] with shapes 20
8
filling: person[M08][10][08][001] with shapes 20
filling: person[M08][10][08][002] with shapes 20
filling: person[M08][10][08][003] with shapes 20
filling: person[M08][10][08][004] with shapes 20
filling: person[M08][10][08][005] with shapes 20
filling: person[M08][10][08][006] with shapes 20
filling: person[M08][10][08][007] with shapes 20
filling: person[M08][10][08][008] with shapes 20
filling: person[M08][10][08][009] with shapes 20
filling: person[M08][10][08][010] with shapes 20
filling: person[M08][10][08][011] with shapes 20
filling: person[M08][10][08][012] with shapes 20
9
filling: person[M08][10][09][001] with shapes 20
filling: person[M08][10][09][002] with shapes 20
filling: person[M08][10][09][003] with shapes 20
filling: person[

In [18]:
linpDir = os.listdir(inpDir)
for personStr1 in linpDir:
    pi += 1
    print("person: %s [%d/%d]" % (personStr1,pi,len(linpDir)))
    
    personFolder = '%s/%s/words' % (inpDir,personStr1)
    lpersonFolder = os.listdir(personFolder)
    si = 0
    for wordId in lpersonFolder:
        si += 1
        c=0
        person[personStr1][wordId] = {}
        print("reading sentence %s for person %s [%d/%d,%d/%d]" % (wordId,personStr1,si,len(lpersonFolder),pi,len(linpDir)))
        wordFolder = '%s/%s' % (personFolder,wordId)
        for wordId2 in os.listdir(wordFolder):
            wordFolder2 = '%s/%s/%s' % (personFolder,wordId,wordId2)
            person[personStr1][wordId][wordId2] = {}
            c=c+1
            print(c)
            for frame1 in os.listdir(wordFolder2):
                file = "%s/%s" % (wordFolder2,frame1)
                if(not os.path.isfile(file)):
                    print("%s does not exist" % file)
                    sys.exit(1)
                if(frame1[0:5] != "color"):
                    # skip depth data
                    
                    continue
                frame1 = frame1[6:-4]
                #print("reading file: %s" % file)
                img = cv2.imread(file)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                #speed up detector by resizing first
                img2 = cv2.resize(img, None, fx=ratio, fy=ratio)

                
                #detects whole face
                rects = detector(img2, 1)
                if len(rects) == 0:
                    print("error finding head at file: %s" % file)
                    continue
                rects[0] = dlib.scale_rect(rects[0],1/ratio)
                shape1 = predictor(image=img,box=rects[0])
                shape1 = shape_to_np(shape1)
                
                #indices from 48 to 67 are for mouth
                shape1 = [shape1[x] for x in range(48,68)]
                (x, y, w, h) = get_rect(shape1)

                for s in shape1:
                    s[0] -= x
                    s[1] -= y
                
                person[personStr1][wordId][wordId2][frame1] = {}
                if firstImg is None:
                    firstImg = img
                    firstPerson = personStr1
                    firstSentenceId = wordId
                    firstSentenceId2 = wordId2
                if ((firstPerson == personStr1) and (firstSentenceId == wordId) and (firstSentenceId2 == wordId2)):
                    mouth1 = img[y:y+h, x:x+w].copy()
                    
                
                
                print("filling: person[%s][%s][%s][%s] with shapes %d" % (personStr1,wordId,wordId2,frame1,len(shape1)))
                person[personStr1][wordId][wordId2][frame1]["mouth1"] = mouth1
                
                person[personStr1][wordId][wordId2][frame1]["shape1"] = shape1
            
        #for now just one sentence
       
        #if SHORT_TEST:
            #break
    #for now just one person
    #if SHORT_TEST:
        #break

person: M08 [22/15]
reading sentence 01 for person F01 [1/10,22/15]
1
filling: person[F01][01][01][001] with shapes 20
filling: person[F01][01][01][002] with shapes 20
filling: person[F01][01][01][003] with shapes 20
filling: person[F01][01][01][004] with shapes 20
filling: person[F01][01][01][005] with shapes 20
filling: person[F01][01][01][006] with shapes 20
filling: person[F01][01][01][007] with shapes 20
filling: person[F01][01][01][008] with shapes 20
filling: person[F01][01][01][009] with shapes 20
filling: person[F01][01][01][010] with shapes 20
2
filling: person[F01][01][02][001] with shapes 20
filling: person[F01][01][02][002] with shapes 20
filling: person[F01][01][02][003] with shapes 20
filling: person[F01][01][02][004] with shapes 20
filling: person[F01][01][02][005] with shapes 20
filling: person[F01][01][02][006] with shapes 20
filling: person[F01][01][02][007] with shapes 20
3
filling: person[F01][01][03][001] with shapes 20
filling: person[F01][01][03][002] with shape

reading sentence 03 for person F01 [3/10,22/15]
1
filling: person[F01][03][01][001] with shapes 20
filling: person[F01][03][01][002] with shapes 20
filling: person[F01][03][01][003] with shapes 20
filling: person[F01][03][01][004] with shapes 20
filling: person[F01][03][01][005] with shapes 20
filling: person[F01][03][01][006] with shapes 20
filling: person[F01][03][01][007] with shapes 20
filling: person[F01][03][01][008] with shapes 20
filling: person[F01][03][01][009] with shapes 20
filling: person[F01][03][01][010] with shapes 20
filling: person[F01][03][01][011] with shapes 20
filling: person[F01][03][01][012] with shapes 20
2
filling: person[F01][03][02][001] with shapes 20
filling: person[F01][03][02][002] with shapes 20
filling: person[F01][03][02][003] with shapes 20
filling: person[F01][03][02][004] with shapes 20
filling: person[F01][03][02][005] with shapes 20
filling: person[F01][03][02][006] with shapes 20
filling: person[F01][03][02][007] with shapes 20
3
filling: person

filling: person[F01][04][09][003] with shapes 20
filling: person[F01][04][09][004] with shapes 20
filling: person[F01][04][09][005] with shapes 20
filling: person[F01][04][09][006] with shapes 20
filling: person[F01][04][09][007] with shapes 20
filling: person[F01][04][09][008] with shapes 20
10
filling: person[F01][04][10][001] with shapes 20
filling: person[F01][04][10][002] with shapes 20
filling: person[F01][04][10][003] with shapes 20
filling: person[F01][04][10][004] with shapes 20
filling: person[F01][04][10][005] with shapes 20
filling: person[F01][04][10][006] with shapes 20
filling: person[F01][04][10][007] with shapes 20
filling: person[F01][04][10][008] with shapes 20
reading sentence 05 for person F01 [5/10,22/15]
1
filling: person[F01][05][01][001] with shapes 20
filling: person[F01][05][01][002] with shapes 20
filling: person[F01][05][01][003] with shapes 20
filling: person[F01][05][01][004] with shapes 20
filling: person[F01][05][01][005] with shapes 20
filling: person[

filling: person[F01][06][09][004] with shapes 20
filling: person[F01][06][09][005] with shapes 20
filling: person[F01][06][09][006] with shapes 20
filling: person[F01][06][09][007] with shapes 20
filling: person[F01][06][09][008] with shapes 20
10
filling: person[F01][06][10][001] with shapes 20
filling: person[F01][06][10][002] with shapes 20
filling: person[F01][06][10][003] with shapes 20
filling: person[F01][06][10][004] with shapes 20
filling: person[F01][06][10][005] with shapes 20
filling: person[F01][06][10][006] with shapes 20
filling: person[F01][06][10][007] with shapes 20
filling: person[F01][06][10][008] with shapes 20
filling: person[F01][06][10][009] with shapes 20
reading sentence 07 for person F01 [7/10,22/15]
1
filling: person[F01][07][01][001] with shapes 20
filling: person[F01][07][01][002] with shapes 20
filling: person[F01][07][01][003] with shapes 20
filling: person[F01][07][01][004] with shapes 20
filling: person[F01][07][01][005] with shapes 20
filling: person[

9
filling: person[F01][08][09][001] with shapes 20
filling: person[F01][08][09][002] with shapes 20
filling: person[F01][08][09][003] with shapes 20
filling: person[F01][08][09][004] with shapes 20
filling: person[F01][08][09][005] with shapes 20
filling: person[F01][08][09][006] with shapes 20
filling: person[F01][08][09][007] with shapes 20
10
filling: person[F01][08][10][001] with shapes 20
filling: person[F01][08][10][002] with shapes 20
filling: person[F01][08][10][003] with shapes 20
filling: person[F01][08][10][004] with shapes 20
filling: person[F01][08][10][005] with shapes 20
filling: person[F01][08][10][006] with shapes 20
reading sentence 09 for person F01 [9/10,22/15]
1
filling: person[F01][09][01][001] with shapes 20
filling: person[F01][09][01][002] with shapes 20
filling: person[F01][09][01][003] with shapes 20
filling: person[F01][09][01][004] with shapes 20
filling: person[F01][09][01][005] with shapes 20
filling: person[F01][09][01][006] with shapes 20
2
filling: per

...

filling: person[M08][02][03][009] with shapes 20
4
filling: person[M08][02][04][001] with shapes 20
filling: person[M08][02][04][002] with shapes 20
filling: person[M08][02][04][003] with shapes 20
filling: person[M08][02][04][004] with shapes 20
filling: person[M08][02][04][005] with shapes 20
filling: person[M08][02][04][006] with shapes 20
filling: person[M08][02][04][007] with shapes 20
filling: person[M08][02][04][008] with shapes 20
filling: person[M08][02][04][009] with shapes 20
5
filling: person[M08][02][05][001] with shapes 20
filling: person[M08][02][05][002] with shapes 20
filling: person[M08][02][05][003] with shapes 20
filling: person[M08][02][05][004] with shapes 20
filling: person[M08][02][05][005] with shapes 20
filling: person[M08][02][05][006] with shapes 20
filling: person[M08][02][05][007] with shapes 20
6
filling: person[M08][02][06][001] with shapes 20
filling: person[M08][02][06][002] with shapes 20
filling: person[M08][02][06][003] with shapes 20
filling: perso

filling: person[M08][04][03][007] with shapes 20
filling: person[M08][04][03][008] with shapes 20
4
filling: person[M08][04][04][001] with shapes 20
filling: person[M08][04][04][002] with shapes 20
filling: person[M08][04][04][003] with shapes 20
filling: person[M08][04][04][004] with shapes 20
filling: person[M08][04][04][005] with shapes 20
filling: person[M08][04][04][006] with shapes 20
filling: person[M08][04][04][007] with shapes 20
5
filling: person[M08][04][05][001] with shapes 20
filling: person[M08][04][05][002] with shapes 20
filling: person[M08][04][05][003] with shapes 20
filling: person[M08][04][05][004] with shapes 20
filling: person[M08][04][05][005] with shapes 20
filling: person[M08][04][05][006] with shapes 20
filling: person[M08][04][05][007] with shapes 20
6
filling: person[M08][04][06][001] with shapes 20
filling: person[M08][04][06][002] with shapes 20
filling: person[M08][04][06][003] with shapes 20
filling: person[M08][04][06][004] with shapes 20
filling: perso

filling: person[M08][06][03][007] with shapes 20
filling: person[M08][06][03][008] with shapes 20
filling: person[M08][06][03][009] with shapes 20
filling: person[M08][06][03][010] with shapes 20
4
filling: person[M08][06][04][001] with shapes 20
filling: person[M08][06][04][002] with shapes 20
filling: person[M08][06][04][003] with shapes 20
filling: person[M08][06][04][004] with shapes 20
filling: person[M08][06][04][005] with shapes 20
filling: person[M08][06][04][006] with shapes 20
filling: person[M08][06][04][007] with shapes 20
filling: person[M08][06][04][008] with shapes 20
5
filling: person[M08][06][05][001] with shapes 20
filling: person[M08][06][05][002] with shapes 20
filling: person[M08][06][05][003] with shapes 20
filling: person[M08][06][05][004] with shapes 20
filling: person[M08][06][05][005] with shapes 20
filling: person[M08][06][05][006] with shapes 20
filling: person[M08][06][05][007] with shapes 20
filling: person[M08][06][05][008] with shapes 20
filling: person[

filling: person[M08][08][03][007] with shapes 20
filling: person[M08][08][03][008] with shapes 20
filling: person[M08][08][03][009] with shapes 20
4
filling: person[M08][08][04][001] with shapes 20
filling: person[M08][08][04][002] with shapes 20
filling: person[M08][08][04][003] with shapes 20
filling: person[M08][08][04][004] with shapes 20
filling: person[M08][08][04][005] with shapes 20
filling: person[M08][08][04][006] with shapes 20
filling: person[M08][08][04][007] with shapes 20
filling: person[M08][08][04][008] with shapes 20
filling: person[M08][08][04][009] with shapes 20
5
filling: person[M08][08][05][001] with shapes 20
filling: person[M08][08][05][002] with shapes 20
filling: person[M08][08][05][003] with shapes 20
filling: person[M08][08][05][004] with shapes 20
filling: person[M08][08][05][005] with shapes 20
filling: person[M08][08][05][006] with shapes 20
filling: person[M08][08][05][007] with shapes 20
filling: person[M08][08][05][008] with shapes 20
6
filling: perso

filling: person[M08][10][01][001] with shapes 20
filling: person[M08][10][01][002] with shapes 20
filling: person[M08][10][01][003] with shapes 20
filling: person[M08][10][01][004] with shapes 20
filling: person[M08][10][01][005] with shapes 20
filling: person[M08][10][01][006] with shapes 20
2
filling: person[M08][10][02][001] with shapes 20
filling: person[M08][10][02][002] with shapes 20
filling: person[M08][10][02][003] with shapes 20
filling: person[M08][10][02][004] with shapes 20
filling: person[M08][10][02][005] with shapes 20
filling: person[M08][10][02][006] with shapes 20
3
filling: person[M08][10][03][001] with shapes 20
filling: person[M08][10][03][002] with shapes 20
filling: person[M08][10][03][003] with shapes 20
filling: person[M08][10][03][004] with shapes 20
filling: person[M08][10][03][005] with shapes 20
filling: person[M08][10][03][006] with shapes 20
filling: person[M08][10][03][007] with shapes 20
4
filling: person[M08][10][04][001] with shapes 20
filling: perso

COLS = 3
plt.figure(figsize=(10,20))
plt.subplot(10,COLS,1)
plt.imshow(firstImg,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
for i in range(1,10):
    id = '%03d' % i
    if not id in person[firstPerson][firstSentenceId][firstSentenceId2]:
        print("%s not found!" % id)
    else:
        mouth = person[firstPerson][firstSentenceId][firstSentenceId2][id]["mouth"]
        shape = person[firstPerson][firstSentenceId][firstSentenceId2][id]["shape"]
        plt.subplot(10,COLS,COLS+1+(i-1)*COLS)
        plt.imshow(mouth,cmap = 'gray')
        plt.title('Mouth Image')
        plt.xticks([])
        plt.yticks([])
        
        mouthEdge = np.ones_like(mouth)
        plt.subplot(10,COLS,COLS+2+(i-1)*COLS)
        for (x, y) in shape:
            cv2.circle(mouthEdge, (x, y), 1, (0, 0, 255), -1)
        plt.imshow(mouthEdge,cmap = 'gray')
        plt.title('Mouth Image Edge')
        plt.xticks([])
        plt.yticks([])

        both = np.copy(mouth)
        plt.subplot(10,COLS,COLS+3+(i-1)*COLS)
        for (x, y) in shape:
            cv2.circle(both, (x, y), 1, (0, 0, 255), -1)
        plt.imshow(both,cmap = 'gray')
        plt.title('Both')
        plt.xticks([])
        plt.yticks([])
plt.show()

In [25]:
fileCount = 0
for p in person.keys():
    for sentenceId in person[p].keys():
        shapes = []
        for sentenceId2 in person[p][sentenceId].keys():
            for id in person[p][sentenceId][sentenceId2].keys():
                shape = person[p][sentenceId][sentenceId2][id]["shape"]
                for (x,y) in shape:
                    shapes.append((id,sentenceId2,x,y))
        df = pd.DataFrame(data = shapes,columns = ['id','sentenceId2','x','y'])
        path = "%s/phrases/%s" % (p,sentenceId)
        try:
            os.makedirs(path)
        except FileExistsError as e:
            pass
        filename = "shapes_p.csv"
        csvPath = "%s/%s" % (path,filename)
        print("saving %s" % csvPath)
        print(df)
        df.to_csv(csvPath)
        fileCount += 1
#total fileCount needs to be below 500
print("fileCount: %d" % fileCount)

{'01': {'001': {'mouth1': array([[ 94,  94, 101, 100, 101,  98,  98,  88,  80,  76,  77,  86,  95,
         98, 101,  95,  93,  90,  90,  94, 114, 112, 119, 118, 120, 121,
        114, 115],
       [ 87,  86,  94,  93,  93,  87,  87,  80,  73,  70,  68,  68,  70,
         73,  80,  72,  78,  77,  77,  77,  89,  90, 104, 104, 117, 118,
        109, 107],
       [ 84,  80,  87,  83,  81,  72,  71,  59,  55,  54,  54,  56,  59,
         61,  66,  65,  70,  67,  64,  64,  59,  69,  87,  86, 102, 100,
         95,  99],
       [ 73,  66,  65,  60,  47,  37,  45,  40,  40,  41,  50,  41,  47,
         52,  61,  61,  61,  59,  58,  54,  49,  50,  56,  48,  64,  65,
         77,  92],
       [ 60,  49,  44,  44,  35,  33,  42,  47,  48,  55,  52,  52,  49,
         56,  60,  62,  67,  70,  69,  66,  62,  58,  57,  51,  46,  49,
         50,  59],
       [ 46,  25,  25,  24,  32,  34,  49,  57,  62,  71,  65,  71,  66,
         71,  71,  70,  78,  81,  79,  78,  74,  66,  60,  54,  42,  36,
   

saving F01/phrases/08/shapes.csv
       id sentenceId2   x  y
0     001          01   0  7
1     001          01   5  3
2     001          01  10  0
3     001          01  14  1
4     001          01  17  0
...   ...         ...  .. ..
1495  006          10  17  5
1496  006          10  25  6
1497  006          10  17  5
1498  006          10  13  5
1499  006          10  10  5

[1500 rows x 4 columns]
saving F01/phrases/09/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   5  2
2     001          01  11  0
3     001          01  14  1
4     001          01  18  0
...   ...         ...  .. ..
1435  007          10  17  5
1436  007          10  25  5
1437  007          10  17  5
1438  007          10  14  5
1439  007          10  11  5

[1440 rows x 4 columns]
saving F01/phrases/10/shapes.csv
       id sentenceId2   x  y
0     001          01   0  6
1     001          01   5  2
2     001          01  10  0
3     001          01  13  0
4     001 

saving F04/phrases/10/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   4  2
2     001          01   9  0
3     001          01  12  1
4     001          01  15  0
...   ...         ...  .. ..
2315  010          10  15  4
2316  010          10  22  4
2317  010          10  15  4
2318  010          10  12  4
2319  010          10   9  4

[2320 rows x 4 columns]
saving F05/phrases/01/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   6  2
2     001          01  11  0
3     001          01  14  1
4     001          01  18  0
...   ...         ...  .. ..
2535  013          10  17  4
2536  013          10  27  4
2537  013          10  17  5
2538  013          10  13  5
2539  013          10   9  5

[2540 rows x 4 columns]
saving F05/phrases/02/shapes.csv
       id sentenceId2   x  y
0     001          01   0  4
1     001          01   6  2
2     001          01  11  0
3     001          01  15  1
4     001 

saving F07/phrases/02/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   6  2
2     001          01  11  0
3     001          01  14  2
4     001          01  17  0
...   ...         ...  .. ..
2095  010          10  18  4
2096  010          10  27  4
2097  010          10  18  5
2098  010          10  15  5
2099  010          10  11  5

[2100 rows x 4 columns]
saving F07/phrases/03/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   4  2
2     001          01   9  0
3     001          01  13  1
4     001          01  16  0
...   ...         ...  .. ..
2535  013          10  17  4
2536  013          10  25  5
2537  013          10  17  5
2538  013          10  13  5
2539  013          10  10  5

[2540 rows x 4 columns]
saving F07/phrases/04/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   5  2
2     001          01   9  0
3     001          01  12  1
4     001 

saving F09/phrases/03/shapes.csv
       id sentenceId2   x  y
0     001          01   0  7
1     001          01   5  3
2     001          01  10  0
3     001          01  14  1
4     001          01  18  0
...   ...         ...  .. ..
1775  010          10  17  5
1776  010          10  28  4
1777  010          10  17  5
1778  010          10  13  5
1779  010          10   9  5

[1780 rows x 4 columns]
saving F09/phrases/04/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   5  2
2     001          01   9  0
3     001          01  13  1
4     001          01  17  0
...   ...         ...  .. ..
1955  009          10  16  4
1956  009          10  25  4
1957  009          10  16  5
1958  009          10  12  5
1959  009          10   9  5

[1960 rows x 4 columns]
saving F09/phrases/05/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   5  2
2     001          01   9  0
3     001          01  13  1
4     001 

saving F11/phrases/05/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   4  2
2     001          01   9  0
3     001          01  13  1
4     001          01  16  0
...   ...         ...  .. ..
1395  007          10  18  4
1396  007          10  27  6
1397  007          10  18  5
1398  007          10  14  5
1399  007          10  11  4

[1400 rows x 4 columns]
saving F11/phrases/06/shapes.csv
       id sentenceId2   x  y
0     001          01   0  4
1     001          01   5  2
2     001          01  10  0
3     001          01  13  1
4     001          01  17  0
...   ...         ...  .. ..
1335  006          10  17  4
1336  006          10  25  6
1337  006          10  17  6
1338  006          10  13  6
1339  006          10  10  5

[1340 rows x 4 columns]
saving F11/phrases/07/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   5  2
2     001          01  10  0
3     001          01  13  1
4     001 

saving M02/phrases/07/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   5  2
2     001          01  10  0
3     001          01  13  1
4     001          01  17  0
...   ...         ...  .. ..
2115  011          10  17  3
2116  011          10  24  4
2117  011          10  17  3
2118  011          10  14  4
2119  011          10  10  4

[2120 rows x 4 columns]
saving M02/phrases/08/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   3  2
2     001          01   7  0
3     001          01  11  1
4     001          01  14  0
...   ...         ...  .. ..
3035  014          10  17  3
3036  014          10  25  4
3037  014          10  17  4
3038  014          10  14  5
3039  014          10  10  4

[3040 rows x 4 columns]
saving M02/phrases/09/shapes.csv
       id sentenceId2   x  y
0     001          01   0  4
1     001          01   5  1
2     001          01  10  0
3     001          01  13  1
4     001 

saving M08/phrases/01/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   5  2
2     001          01  10  0
3     001          01  13  1
4     001          01  16  0
...   ...         ...  .. ..
1715  008          10  17  3
1716  008          10  25  3
1717  008          10  17  3
1718  008          10  14  4
1719  008          10  10  4

[1720 rows x 4 columns]
saving M08/phrases/02/shapes.csv
       id sentenceId2   x  y
0     001          01   0  4
1     001          01   6  2
2     001          01  11  0
3     001          01  15  1
4     001          01  18  0
...   ...         ...  .. ..
1855  010          10  16  3
1856  010          10  24  3
1857  010          10  16  3
1858  010          10  13  4
1859  010          10   9  4

[1860 rows x 4 columns]
saving M08/phrases/03/shapes.csv
       id sentenceId2   x  y
0     001          01   0  5
1     001          01   5  2
2     001          01   9  0
3     001          01  12  1
4     001 

In [ ]:
fileCount = 0
for p in person.keys():
    for wordId in person[p].keys():
        shapes_w = []
        for wordId2 in person[p][wordId].keys():
            for id in person[p][wordId][wordId2].keys():
                shape1 = person[p][wordId][wordId2][id]["shape1"]
                for (x,y) in shape1:
                    shapes_w.append((id,wordId2,x,y))
        df = pd.DataFrame(data = shapes_w,columns = ['id','wordId2','x','y'])
        path = "%s/words/%s" % (p,wordId)
        try:
            os.makedirs(path)
        except FileExistsError as e:
            pass
        filename = "shapes_w.csv"
        csvPath = "%s/%s" % (path,filename)
        print("saving %s" % csvPath)
        print(df)
        df.to_csv(csvPath)
        fileCount += 1
#total fileCount needs to be below 500
print("fileCount: %d" % fileCount)